In [0]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import STL10
import os
import numpy as np
import torch.nn.functional as F
from torch import optim

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
## network
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        #encoder layer
        self.enc_conv1 = nn.Conv2d(3, 64, 3, stride=2, padding=1)
        self.enc_bn1 = nn.BatchNorm2d(64)
        self.enc_conv2 = nn.Conv2d(64, 128, 3, stride=2, padding=1)
        self.enc_bn2 = nn.BatchNorm2d(128)
        self.enc_conv3 = nn.Conv2d(128, 256, 3, stride=2, padding=1)
        self.enc_bn3 = nn.BatchNorm2d(256)
        self.enc_conv4 = nn.Conv2d(256, 512, 3, stride=2, padding=1)
        self.enc_bn4 = nn.BatchNorm2d(512)
        #decoder layer
        self.dec_conv1 = nn.ConvTranspose2d(512, 256, 3, stride=2, padding=1,output_padding=1)
        self.dec_bn1 = nn.BatchNorm2d(256)
        self.dec_conv2 = nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1,output_padding=1)
        self.dec_bn2 = nn.BatchNorm2d(128)
        self.dec_conv3 = nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1,output_padding=1)
        self.dec_bn3 = nn.BatchNorm2d(64)
        self.dec_conv4 = nn.ConvTranspose2d(64, 3, 3, stride=2, padding=1,output_padding=1)
        
        
        
    def encoder(self,x):
      x = F.relu(self.enc_conv1(x))
      x = self.enc_bn1(x)
      x = F.relu(self.enc_conv2(x))
      x = self.enc_bn2(x)
      x = F.relu(self.enc_conv3(x))
      x = self.enc_bn3(x)
      x = F.relu(self.enc_conv4(x))
      x = self.enc_bn4(x)
      return x
    
    def decoder(self,x):
      x = F.relu(self.dec_conv1(x))
      x = self.dec_bn1(x)
      x = F.relu(self.dec_conv2(x))
      x = self.dec_bn2(x)
      x = F.relu(self.dec_conv3(x))
      x = self.dec_bn3(x)
      x = F.sigmoid(self.dec_conv4(x))
      return x
      


    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [0]:
if not os.path.exists('./gdrive/My Drive/Colab Notebooks/denoise'):
    os.mkdir('./gdrive/My Drive/Colab Notebooks/denoise')

    

num_epochs = 100
batch_size = 32
learning_rate = 1e-4

img_transform = transforms.Compose([#transforms.Resize(96),
    transforms.ToTensor()
    
])

dataset = STL10('./data',  split='unlabeled',transform=img_transform,download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)



model = autoencoder()
model.cuda()

from collections import OrderedDict

'''
state_dict = torch.load('./gdrive/My Drive/Colab Notebooks/vae.pth')
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k
    #name = k[7:] # remove "module."
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)

if torch.cuda.is_available():
    model.cuda()
'''

loss_function = nn.MSELoss()



optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        size=img.size()
        noise=torch.Tensor(size).normal_(0,1)
        noised_img=img+noise
        noised_img=noised_img.type(torch.FloatTensor)
        noised_img = Variable(noised_img)
        img=Variable(img)
        
        if torch.cuda.is_available():
            noised_img = noised_img.cuda()
            img=img.cuda()
            
        optimizer.zero_grad()
        recon_batch  = model(noised_img)
        loss = loss_function(recon_batch, img)
        loss.backward()
        train_loss += loss.data.item()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(img),
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                loss.data.item() / len(img)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(dataloader.dataset)))
    if epoch % 5 == 0:
        img=img.cpu().data
        noised_img=noised_img.cpu().data
        save=recon_batch.cpu().data
        noised_img.clamp(0,1)
        save_image(img, './gdrive/My Drive/Colab Notebooks/denoise/GT_{}.png'.format(epoch))
        save_image(noised_img, './gdrive/My Drive/Colab Notebooks/denoise/noised_{}.png'.format(epoch))
        save_image(save, './gdrive/My Drive/Colab Notebooks/denoise/output_{}.png'.format(epoch))

        torch.save(model.state_dict(), './gdrive/My Drive/Colab Notebooks/denoise/dae.pth')

Files already downloaded and verified


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 0 [0/100000 (0%)]	Loss: 0.002956
Train Epoch: 0 [16000/100000 (16%)]	Loss: 0.000495
Train Epoch: 0 [32000/100000 (32%)]	Loss: 0.000485
Train Epoch: 0 [48000/100000 (48%)]	Loss: 0.000439
Train Epoch: 0 [64000/100000 (64%)]	Loss: 0.000447
Train Epoch: 0 [80000/100000 (80%)]	Loss: 0.000406
Train Epoch: 0 [96000/100000 (96%)]	Loss: 0.000431
====> Epoch: 0 Average loss: 0.0005
Train Epoch: 1 [0/100000 (0%)]	Loss: 0.000443
Train Epoch: 1 [16000/100000 (16%)]	Loss: 0.000477
Train Epoch: 1 [32000/100000 (32%)]	Loss: 0.000386
Train Epoch: 1 [48000/100000 (48%)]	Loss: 0.000367
Train Epoch: 1 [64000/100000 (64%)]	Loss: 0.000388
Train Epoch: 1 [80000/100000 (80%)]	Loss: 0.000374
Train Epoch: 1 [96000/100000 (96%)]	Loss: 0.000409
====> Epoch: 1 Average loss: 0.0004
Train Epoch: 2 [0/100000 (0%)]	Loss: 0.000414
Train Epoch: 2 [16000/100000 (16%)]	Loss: 0.000509
Train Epoch: 2 [32000/100000 (32%)]	Loss: 0.000400
Train Epoch: 2 [48000/100000 (48%)]	Loss: 0.000403
Train Epoch: 2 [64000/100

In [0]:
def psnr(img1, img2):
    mse = np.mean( (img1 - img2) ** 2 )
    if mse == 0:
      return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

In [27]:
from PIL import Image
import math

import cv2
img1=cv2.imread('./gdrive/My Drive/Colab Notebooks/denoise/GT_75.png')
img2=cv2.imread('./gdrive/My Drive/Colab Notebooks/denoise/output_75.png')
img3=cv2.imread('./gdrive/My Drive/Colab Notebooks/denoise/noised_75.png')

psnr1=psnr(img1,img2)
psnr2=psnr(img1,img3)
print('psnr denoised : ',psnr1)
print('psnr noised : ',psnr2)

psnr denoised :  29.15316780925073
psnr noised :  28.273123568053013
